# Measurements type Loading

Notebook to load Measurements type data into the databse

## Imports

In [1]:
import psycopg2
from tabulate import tabulate
#import a folder in the parent directory
import sys
sys.path.append('../')
import queries.queries as qrs

## Connection

In [ ]:
try:
    # Connect to the PostgreSQL database
    conn = qrs.connect()
    print("Connected to the database")

except (Exception, psycopg2.DatabaseError) as error:
    print(error)

Connected to the database


## Measurement type data

We have to add the parameters of the sample.

The name is a compulsory parameter, it has to be always filled.

The description is not compulsory but is recommended.

In [3]:
name = 'UT_2025_1'

description = 'UT measurements, 2025 metodology. Measuring at the highest z resolution, and with the focal zone in the frontwall.'

main_parameters = {'name': name, 'description': description}

Then metadata parameters are optional and can vary depending on the measurement type so its loading to the database is more dinamic.

The metadata list can be empty.

To load metadata parameters we have to fill a list with dictionaries, the dictionaries may have this parameters:

1. key: The name of the parameter

2. value: The value of the parameter

3. type: The units of the parameter

Here we include an example to save the technique, Ultrasound Pulse Echo which type is nominal

metadata_parameters.append({'key':'technique', 'value':'Ultrasound Pulse Echo', 'type':'nominal' })


In [ ]:
# metadata

#for each parameter of the measurement a metadata has to be created

metadata_parameters = []

#technique
metadata_parameters.append({'key':'technique', 'value':'Ultrasound Pulse Echo', 'type':'nominal' })

#Nominal Frecuency
metadata_parameters.append({'key':'Nominal Frequency', 'value':'5', 'type':'Mhz'})

#transducer diameter
metadata_parameters.append({'key':'transducer diameter', 'value':'9.529', 'type':'mm'})

#Z Resolution
metadata_parameters.append({'key':'Z Resolution', 'value':'0.05940588', 'type':'mm'})

#Z range
metadata_parameters.append({'key':'Z range', 'value':'10', 'type':'mm'})

#X Resolution
metadata_parameters.append({'key':'X Resolution', 'value':'1', 'type':'mm'})

#Y Resolution
metadata_parameters.append({'key':'Y Resolution', 'value':'1', 'type':'mm'})

#sample transducer distance
metadata_parameters.append({'key':'sample transducer distance', 'value':'77', 'type':'mm'})

#gain
metadata_parameters.append({'key':'gain', 'value':'11', 'type':'dB'})

#sound velocity in water
metadata_parameters.append({'key':'sound velocity in water', 'value':'1500', 'type':'m/s'})

#sound velocity in sample
metadata_parameters.append({'key':'sound velocity in sample', 'value':'2970', 'type':'m/s'})

#transducer movement speed
metadata_parameters.append({'key':'transducer movement speed', 'value':'20', 'type':'mm/s'})

## Load into the table

In [5]:
print('Parameters to be inserted: ')
for key, value in main_parameters.items():
    print(f"    -- {key}: {value}")

Parameters to be inserted: 
    -- name: UT_2025_1
    -- description: UT measurements, 2025 metodology. Measuring at the highest z resolution, and with the focal zone in the frontwall.


In [6]:
table_name = 'measurementtypes'

# Extract column names and values from the attributes dictionary
columns = ', '.join(main_parameters.keys())
values = ', '.join([f"'{v}'" for v in main_parameters.values()])

# Construct the SQL INSERT statement
sql = f"INSERT INTO {table_name} ({columns}) VALUES ({values})"

print(sql)

# Create a cursor object using the cursor() method
cursor = conn.cursor()

# Execute the SQL statement
cursor.execute(sql)

cursor.execute('COMMIT')

cursor.close()

INSERT INTO measurementtypes (name, description) VALUES ('UT_2025_1', 'UT measurements, 2025 metodology. Measuring at the highest z resolution, and with the focal zone in the frontwall.')


## Make sure insert was correct

In [ ]:
data = qrs.get_data(table_name)

data

,id_measurementtype,name_measurementtype,description_measurementtype
0,1,UT_2025,"UT measurements, 2025 metodology. The ideal co..."
1,2,XCT_2024,"First XCT measurements, 2024 metodology."
2,3,UT_2024,"UT measurements, 2024 metodology. Used for the..."
3,4,UT_2025_1,"UT measurements, 2025 metodology. Measuring at..."


Now we store the id of the inserted row, to insert its related metadata

In [8]:
row_id = data['id_' + table_name[:-1]].values[-1]

print(f"Row ID: {row_id}")

Row ID: 4


## Load into the metadata table

In [9]:
metadata_table_name =  table_name[:-1] + '_metadata'

for attributes in metadata_parameters:

    attributes[table_name[:-1] + '_id'] = row_id

    # Extract column names and values from the attributes dictionary
    columns = ', '.join(attributes.keys())
    values = ', '.join([f"'{v}'" for v in attributes.values()])

    # Construct the SQL INSERT statement
    sql = f"INSERT INTO {metadata_table_name} ({columns}) VALUES ({values})"

    print(sql)

    # Create a cursor object using the cursor() method
    cursor = conn.cursor()

    # Execute the SQL statement
    cursor.execute(sql)

    cursor.execute('COMMIT')

    cursor.close()

INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('technique', 'Ultrasound Pulse Echo', 'nominal', '4')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('Nominal Frecuency', '5', 'Mhz', '4')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('transducer diameter', '9.529', 'mm', '4')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('Z Resolution', '0.05940588', 'mm', '4')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('Z range', '10', 'mm', '4')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('X Resolution', '1', 'mm', '4')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('Y Resolution', '1', 'mm', '4')
INSERT INTO measurementtype_metadata (key, value, type, measurementtype_id) VALUES ('sample transducer distance', '77', 'mm', '4')
INSERT INTO m

## Make sure insert was correct

In [ ]:
data = qrs.get_data_metadata(table_name)

data

,id_measurementtype,name_measurementtype,description_measurementtype,technique_measurementtype,Nominal Frecuency_measurementtype,Z Resolution_measurementtype,X Resolution_measurementtype,Y Resolution_measurementtype,sample transducer distance_measurementtype,transducer z position_measurementtype,...,resolution_measurementtype,frames_measurementtype,exposure time_measurementtype,projections_measurementtype,target_measurementtype,filter_measurementtype,binning_measurementtype,equipment_measurementtype,Z range_measurementtype,transducer diameter_measurementtype
0,1,UT_2025,"UT measurements, 2025 metodology. The ideal co...",Ultrasound Pulse Echo nominal,5 Mhz,0.02232141 mm,1 mm,1 mm,77 mm,120 mm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,XCT_2024,"First XCT measurements, 2024 metodology.",X Ray Computed Tomography nominal,NaN,NaN,NaN,NaN,NaN,NaN,...,20 um,5 cardinal,0.2 s,3300 cardinal,W nominal,LE1 nominal,1 cardinal,Versa nominal,NaN,NaN
2,3,UT_2024,"UT measurements, 2024 metodology. Used for the...",Ultrasound Pulse Echo nominal,5 Mhz,0.05940588 mm,1 mm,1 mm,70 mm,126 mm,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30 mm,NaN
3,4,UT_2025_1,"UT measurements, 2025 metodology. Measuring at...",Ultrasound Pulse Echo nominal,5 Mhz,0.05940588 mm,1 mm,1 mm,77 mm,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10 mm,9.529 mm


In [11]:
conn.close()